# Hospital Ownership and Quality of Care

## Aim of analysis

To assess which ownership group has the highest proportion of its hospitals with overall quality of care better than the national average

### 1. Processing relevant data from database

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from functools import reduce
pd.options.mode.chained_assignment = None 

conn = sqlite3.connect("Hospital.db")
c = conn.cursor()

hospital_df = pd.read_sql_query('''
SELECT hospital_info.provider_id, hospital_info.hospital_name, hospital_info.ownership, hospital_info.rating,
   quality_of_care.mortality_nat_comparison, quality_of_care.safety_of_care_nat_comparison, 
   quality_of_care.readmission_nat_comparison, quality_of_care.patient_experience_nat_comparison,
   quality_of_care.effectiveness_of_care_nat_comparison, quality_of_care.timeliness_of_care_nat_comparison,
   quality_of_care.efficient_imaging_nat_comparison
FROM hospital_info
LEFT JOIN quality_of_care on hospital_info.provider_id = quality_of_care.provider_id
''', conn)

conn.close()

### 2. Data information

In [ ]:
hospital_df.head()

In [ ]:
hospital_df.info()

In [ ]:
hospital_df.shape

#### 2.1 Displaying and visualising the average rating of each ownership group

In [ ]:
avg_rating = hospital_df.groupby('ownership')['rating'].mean()
avg_rating.sort_values(ascending = False)

In [ ]:
avg_rating.plot(kind = 'bar')

### 3. Cleaning data

#### 3.1 Dealing with missing data

In [ ]:
hospital_df.replace("Not Available", np.nan, inplace=True)

hospital_df.dropna(how='all', inplace=True)

#### 3.2 Removing hospital ownerships with small sample size

In [ ]:
df2 = hospital_df.groupby('ownership')['ownership'].count()
df200 = df2[df2 < 200]

df200

In [ ]:
remove_owners = ['Government - Federal', 'Government - State', 'Physician', 'Tribal']

clean_data = hospital_df[hospital_df.ownership.isin(remove_owners) == False]

#### 3.3 Displaying the number of hospitals owned by the remaining groups

In [ ]:
count_owners = clean_data.groupby('ownership')['ownership'].count()
count_owners

#### 3.4 Visualising the number of hospitals owned by each group

In [ ]:
count_owners.plot(kind='pie')

### 4. Data analysis

#### 4.1 Calculating the proportion of each quality of care measurement below, above and same as the national average grouped by hospital ownership 

In [ ]:
pat_exp_df = clean_data.groupby('ownership')['patient_experience_nat_comparison'].value_counts(normalize=True)
mort_df = clean_data.groupby('ownership')['mortality_nat_comparison'].value_counts(normalize=True)
soc_df = clean_data.groupby('ownership')['safety_of_care_nat_comparison'].value_counts(normalize=True)
read_df = clean_data.groupby('ownership')['readmission_nat_comparison'].value_counts(normalize=True)
eoc_df = clean_data.groupby('ownership')['effectiveness_of_care_nat_comparison'].value_counts(normalize=True)
time_df = clean_data.groupby('ownership')['timeliness_of_care_nat_comparison'].value_counts(normalize=True)
eff_img_df = clean_data.groupby('ownership')['efficient_imaging_nat_comparison'].value_counts(normalize=True)

n1 = pat_exp_df.reset_index(name = 'pat_exp')
n2 = soc_df.reset_index(name = 'soc')
n3 = eoc_df.reset_index(name = 'eoc')
n4 = time_df.reset_index(name = 'time')
n5 = eff_img_df.reset_index(name = 'eff_img')

n6 = mort_df.reset_index(name = 'mort')
n7 = read_df.reset_index(name = 'read')

n1

#### 4.2 Selecting proportions for 5 quality of care measurements that are above the national average

In [ ]:
above1 = n1[n1['patient_experience_nat_comparison'] == "Above the national average"]
above2 = n2[n2['safety_of_care_nat_comparison'] == "Above the national average"]
above3 = n3[n3['effectiveness_of_care_nat_comparison'] == "Above the national average"]
above4 = n4[n4['timeliness_of_care_nat_comparison'] == "Above the national average"]
above5 = n5[n5['efficient_imaging_nat_comparison'] == "Above the national average"]

above1

#### 4.3 Selecting proportions for the 2 remaining quality of care measurements that are below the national average 

In [ ]:
below6 = n6[n6['mortality_nat_comparison'] == "Below the national average"]
below7 = n7[n7['readmission_nat_comparison'] == "Below the national average"]

below6

#### 4.4 Dropping the national comparison for each quality of care measurement

In [ ]:
above1.drop('patient_experience_nat_comparison', axis=1, inplace=True)
above2.drop('safety_of_care_nat_comparison', axis = 1, inplace=True)
above3.drop('effectiveness_of_care_nat_comparison', axis = 1, inplace=True)
above4.drop('timeliness_of_care_nat_comparison', axis = 1, inplace=True)
above5.drop('efficient_imaging_nat_comparison', axis = 1, inplace=True)

below6.drop('mortality_nat_comparison', axis = 1, inplace=True)
below7.drop('readmission_nat_comparison', axis = 1, inplace=True)

above1

#### 4.5 Merging the quality of care proportion for each measurement into one dataframe  

In [ ]:
# referenced from https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns

data_frames = [above1, above2, above3, above4, above5, below6, below7]

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['ownership'],
                                            how='outer'), data_frames)

df_merged

#### 4.6 Calculating the average quality of care better than the national average  for each hospital group

In [ ]:
df_merged['sum'] = df_merged['pat_exp'] + df_merged['soc']  + df_merged['eoc'] + df_merged['time'] + df_merged['eff_img'] + df_merged['mort'] + df_merged['read'] 

df_merged['quality_of_care'] = df_merged['sum'] / 7

df_merged

#### 4.7 Displaying the final dataframe containing each hospital group and the associated quality of care better than national average proportion

In [ ]:
df_final = df_merged[['ownership','quality_of_care']]

df_final.sort_values(by='quality_of_care', ascending = False)

#### 4.8 Visualising the final result 

In [ ]:
df_final.plot.bar(x='ownership')

### 5. Conclusion 

- hospitals owned by Voluntary non-profit - Church have the highest proportion of its hospitals with overall quality of care better than the national average
- further information will be detailed in the project report 